In [2]:
import numpy as np
import tensorflow as tf
from collections import Counter
import tensorflow_federated as tff
import matplotlib.pyplot as plt
from scipy import stats
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import nest_asyncio
nest_asyncio.apply()

# تنظیم بذر تصادفی برای تکرارپذیری
np.random.seed(42)
tf.random.set_seed(42)

# تنظیم زمینه اجرایی محلی
tff.backends.native.set_local_execution_context()

In [3]:
# بارگذاری داده‌ها
(x_train_m, y_train_m), (x_test_m, y_test_m) = mnist.load_data()

# نرمال‌سازی داده‌ها
x_train_m = x_train_m.astype('float32') / 255.0
x_test_m = x_test_m.astype('float32') / 255.0

# اضافه کردن ابعاد کانال (برای MLP نیازی به این کار نیست، اما برای سازگاری با ورودی‌های مختلف)
x_train_m = x_train_m.reshape((x_train_m.shape[0], -1))  # به شکل (batch_size, 784) تبدیل می‌شود
x_test_m = x_test_m.reshape((x_test_m.shape[0], -1))    # به شکل (batch_size, 784) تبدیل می‌شود

# کدگذاری برچسب‌ها
y_train_m = tf.keras.utils.to_categorical(y_train_m, 10)
y_test_m = tf.keras.utils.to_categorical(y_test_m, 10)



In [4]:
def create_mnist_mlp_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))     # ورودی به شکل مسطح شده (28x28 = 784)
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))     # لایه خروجی با 10 کلاس (softmax)
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])     # کامپایل کردن مدل
    return model
    

In [5]:
def split_data(x, y, num_splits):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    split_indices = np.array_split(indices, num_splits)
    split_data = [(x[indices], y[indices]) for indices in split_indices]  
    return split_data
    
# تقسیم داده‌ها به کلاینت‌ها
num_models = 5
client_data_m = split_data(x_train_m, y_train_m, num_models)

accuracy_no_bug = []  # دقت برای حالت بدون نرمال‌سازی دوباره
accuracy_with_bug = []  # دقت برای حالت با نرمال‌سازی دوباره برای کلاینت 3
client_models_m = []  # لیست برای ذخیره مدل‌های هر کلاینت

# حالت بدون نرمال‌سازی دوباره
for i in range(len(client_data_m)):
    model_data_m = client_data_m[i]
    model_mnist = create_mnist_mlp_model()
    
    # آموزش مدل
    model_mnist.fit(model_data_m[0], model_data_m[1], epochs=5, batch_size=32, verbose=1)
    client_models_m.append(model_mnist)
    
    # ارزیابی دقت مدل بر روی داده‌های تست
    loss, accuracy = model_mnist.evaluate(x_test_m, y_test_m, verbose=0)
    accuracy_no_bug.append(accuracy)  # ذخیره دقت برای حالت بدون نرمال‌سازی دوباره

# حالا، برای آموزش با نرمال‌سازی دوباره برای کلاینت 3
client_models_m = []  # لیست برای ذخیره مدل‌های هر کلاینت (دوباره خالی می‌شود)
for i in range(len(client_data_m)):
    model_data_m = client_data_m[i]
    model_mnist = create_mnist_mlp_model()
    
    # اگر کلاینت شماره 3 باشد، نرمال‌سازی دوباره انجام می‌دهیم
    if i==0: 
        model_data_m = (model_data_m[0] / 255.0, model_data_m[1])
    
    # آموزش مدل
    model_mnist.fit(model_data_m[0], model_data_m[1], epochs=5, batch_size=32, verbose=1)
    client_models_m.append(model_mnist)
    
    # ارزیابی دقت مدل بر روی داده‌های تست
    loss, accuracy = model_mnist.evaluate(x_test_m, y_test_m, verbose=0)
    accuracy_with_bug.append(accuracy)  # ذخیره دقت برای حالت با نرمال‌سازی دوباره

Epoch 1/5
375/375 [==============================] - 2s 6ms/step - loss: 0.4821 - accuracy: 0.8506
Epoch 2/5
375/375 [==============================] - 2s 6ms/step - loss: 0.2120 - accuracy: 0.9363
Epoch 3/5
375/375 [==============================] - 3s 8ms/step - loss: 0.1501 - accuracy: 0.9532
Epoch 4/5
375/375 [==============================] - 2s 5ms/step - loss: 0.1112 - accuracy: 0.9642
Epoch 5/5
375/375 [==============================] - 2s 5ms/step - loss: 0.0921 - accuracy: 0.9698
Epoch 1/5
375/375 [==============================] - 3s 7ms/step - loss: 0.4812 - accuracy: 0.8497
Epoch 2/5
375/375 [==============================] - 3s 7ms/step - loss: 0.2184 - accuracy: 0.9336
Epoch 3/5
375/375 [==============================] - 2s 6ms/step - loss: 0.1530 - accuracy: 0.9532
Epoch 4/5
375/375 [==============================] - 2s 6ms/step - loss: 0.1213 - accuracy: 0.9624
Epoch 5/5
375/375 [==============================] - 3s 7ms/step - loss: 0.0969 - accuracy: 0.9682
Epoch 1/5


In [6]:
for i, acc in enumerate(accuracy_no_bug):
    print(f"Accuracy of client {i+1} without bug: {acc * 100:.2f}%")
print()    
for i, acc in enumerate(accuracy_with_bug):
    print(f"Accuracy of client {i+1} with bug: {acc * 100:.2f}%")


Accuracy of client 1 without bug: 96.01%
Accuracy of client 2 without bug: 96.21%
Accuracy of client 3 without bug: 96.22%
Accuracy of client 4 without bug: 95.37%
Accuracy of client 5 without bug: 96.03%

Accuracy of client 1 with bug: 84.98%
Accuracy of client 2 with bug: 96.18%
Accuracy of client 3 with bug: 96.08%
Accuracy of client 4 with bug: 95.86%
Accuracy of client 5 with bug: 96.12%


### many Samples 

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# تنظیم دانه تصادفی
np.random.seed(42)
tf.random.set_seed(42)

# بارگذاری داده‌ها
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# نرمال‌سازی داده‌ها
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# انتخاب نمونه‌ها
num_samples = 20
sample_indices = np.random.choice(len(x_test), size=num_samples, replace=False)
x_sample = x_test[sample_indices].reshape(-1, 28 * 28)  # Flatten the images
y_sample = y_test[sample_indices]

# پیش‌بینی برای 20 نمونه توسط هر مدل
preds = []
probs = []

for model in client_models_m:
    pred_prob = model.predict(x_sample, verbose=0 )  # پیش‌بینی احتمال‌ها
    pred_class = np.argmax(pred_prob, axis=1)  # کلاس با بیشترین احتمال
    preds.append(pred_class)
    probs.append(pred_prob[np.arange(num_samples), pred_class])  # احتمال کلاس پیش‌بینی‌شده

# نمایش 20 نمونه و پیش‌بینی هر مدل برای آن‌ها
print("True Labels for 20 Samples:")
print(y_sample)  # نمایش برچسب واقعی نمونه‌ها

for i, (pred, prob) in enumerate(zip(preds, probs)):
    print(f"\nPredictions by Model {i + 1}:")
    print(pred)  # نمایش پیش‌بینی هر مدل برای نمونه‌ها
    print("Probability")
    print(prob)  # نمایش احتمال پیش‌بینی‌های صحیح


True Labels for 20 Samples:
[6 2 3 7 2 2 3 4 7 6 6 9 2 0 9 6 8 0 6 5]

Predictions by Model 1:
[6 2 3 7 2 2 3 4 7 6 8 8 2 0 9 6 2 0 6 5]
Probability
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

Predictions by Model 2:
[6 2 3 7 7 2 3 4 7 6 6 9 2 0 9 6 2 0 6 5]
Probability
[0.9998536  0.999785   0.99990964 0.9874592  0.5141855  1.
 0.9998678  0.999997   0.9999999  0.99993026 0.9996505  0.6748664
 0.999998   0.930005   0.99998593 0.9999826  0.93072927 0.99926406
 0.99998474 0.99998593]

Predictions by Model 3:
[6 2 3 7 2 2 3 4 7 6 6 9 2 0 9 6 2 0 6 5]
Probability
[0.9992865  0.9987741  0.9999862  0.9882366  0.9829228  1.
 0.99999976 0.9999306  0.9999267  0.9272349  0.7155002  0.8817752
 0.99999356 0.96667975 0.9997546  0.9744537  0.8992149  0.99997544
 0.99962366 0.9999703 ]

Predictions by Model 4:
[6 2 3 7 2 2 3 4 7 6 6 9 2 0 9 6 2 0 6 5]
Probability
[0.9999994  0.9912287  0.9999949  0.99976903 0.6938817  1.
 0.9999486  0.999997   1.         0.9990759  0.99395704 0.971

In [16]:
def delta_class(models, x_test):
    num_models = len(models)
    preds = [model.predict(x_test).argmax(axis=1) for model in models]   # پیش‌بینی‌ها برای هر مدل
    diffs_matrix = np.zeros((num_models, num_models), dtype=int)    # ماتریس مربعی برای ذخیره تفاوت‌ها
    for i in range(num_models):       # پر کردن ماتریس با تفاوت پیش‌بینی‌ها
        for j in range(i+1, num_models):
            diffs = np.sum(preds[i] != preds[j])  # تعداد تفاوت‌های پیش‌بینی بین مدل i و مدل j
            diffs_matrix[i, j] = diffs
            diffs_matrix[j, i] = diffs  # ماتریس متقارن است
    return diffs_matrix

diff_class = delta_class(client_models_m, x_sample)
print("Difference Matrix for Delta Class:")
print(diff_class)

Difference Matrix for Delta Class:
[[0 3 2 2 2]
 [3 0 1 1 1]
 [2 1 0 0 0]
 [2 1 0 0 0]
 [2 1 0 0 0]]


### اختلاف در احتمال واقعی با در نظر گرفتن کل مقدار

In [9]:
def delta_score_av(models, x_test):
    preds = [model.predict(x_test) for model in models]
    num_models = len(models)
    delta_matrix = np.zeros((num_models, num_models))
    
    for i in range(num_models):
        argmax_i = np.argmax(preds[i], axis=1)
        for j in range(num_models):
            if i != j:
                argmax_j = np.argmax(preds[j], axis=1)

                # محاسبه تعداد اختلافات در argmax
                argmax_diff = np.sum(argmax_i != argmax_j)
                prob_diff = 0
                for k in range(len(argmax_i)):
                    if argmax_i[k] == argmax_j[k]:
                        # احتمال کلاس پیش‌بینی‌شده
                        prob_i = preds[i][k][argmax_i[k]]
                        prob_j = preds[j][k][argmax_j[k]]
                        
                        # بررسی اختلاف در احتمال‌ها
                        if prob_i != prob_j:
                            prob_diff += 1
                
                delta_matrix[i, j] = argmax_diff + prob_diff
    
    return delta_matrix
    
# اجرای تابع با مدل‌ها و نمونه‌ها
diff_score = delta_score_av(client_models_m, x_sample)
print("Distance Matrix for Delta Score:")
print(diff_score)


Distance Matrix for Delta Score:
[[ 0. 19. 19. 18. 18.]
 [19.  0. 19. 18. 19.]
 [19. 19.  0. 19. 19.]
 [18. 18. 19.  0. 18.]
 [18. 19. 19. 18.  0.]]


### اختلاف در مقدار صحیح بدون در نظر گرفتن رقم اعشار

In [10]:
def delta_score_nd(models, x_test):
    # پیش‌بینی‌ها و احتمال‌ها برای هر مدل
    preds = [model.predict(x_test) for model in models]
    # ماتریس برای ذخیره اختلافات
    num_models = len(models)
    delta_matrix = np.zeros((num_models, num_models))
    
    for i in range(num_models):
        argmax_i = np.argmax(preds[i], axis=1)
        for j in range(num_models):
            if i != j:
                argmax_j = np.argmax(preds[j], axis=1)

                # محاسبه تعداد اختلافات در argmax
                argmax_diff = np.sum(argmax_i != argmax_j)

                # محاسبه تعداد اختلافات در احتمال‌ها
                prob_diff = 0
                for k in range(len(argmax_i)):
                    # فقط بررسی می‌کنیم اگر argmax برابر باشد
                    if argmax_i[k] == argmax_j[k]:
                        # گرفتن احتمال‌ها
                        prob_i = preds[i][k]
                        prob_j = preds[j][k]
                        
                        # بررسی اختلاف بدون گرد کردن و محاسبه تعداد تفاوت‌ها
                        if int(prob_i[np.argmax(prob_i)]) != int(prob_j[np.argmax(prob_j)]):
                            prob_diff += 1
                
                # ذخیره اختلافات در ماتریس
                delta_matrix[i, j] = argmax_diff + prob_diff
    
    return delta_matrix

# اجرای تابع با مدل‌ها و نمونه‌ها
diff_score = delta_score_nd(client_models_m, x_sample)
print("Distance Matrix for Delta Score:")
print(diff_score)


Distance Matrix for Delta Score:
[[ 0. 19. 19. 18. 18.]
 [19.  0.  1.  2.  2.]
 [19.  1.  0.  1.  1.]
 [18.  2.  1.  0.  0.]
 [18.  2.  1.  0.  0.]]


### اختلاف با تغییر جایگاه اعشار

In [11]:
def delta_score_d1(models, x_test, decimal_places):
    num_models = len(models)
    deltas = np.zeros((num_models, num_models))

    # پیش‌بینی خروجی‌های هر مدل
    predictions = [model.predict(x_test) for model in models]
    
    # محاسبه‌ی argmax برای هر مدل
    argmax_preds = [np.argmax(pred, axis=1) for pred in predictions]

    # مقایسه مدل‌ها دو به دو
    for i in range(num_models):
        for j in range(i + 1, num_models):
            # مقایسه‌ی argmax ها
            argmax_diff = argmax_preds[i] != argmax_preds[j]
            diff_count = np.sum(argmax_diff)  # شمارش اختلاف در argmax
            
            # مقایسه‌ی احتمالات اگر argmax یکسان باشد
            same_argmax_indices = np.where(argmax_preds[i] == argmax_preds[j])[0]
            
            for idx in same_argmax_indices:
                # مقایسه‌ی احتمالات در همان کلاس‌های argmax
                prob_i = predictions[i][idx][argmax_preds[i][idx]]
                prob_j = predictions[j][idx][argmax_preds[j][idx]]
                
                # استخراج رشته‌ی ارقام اعشار تا تعداد مشخص‌شده
                prob_i_str = str(prob_i)[:decimal_places + 2]  # 2 برای "0."
                prob_j_str = str(prob_j)[:decimal_places + 2]

                # بررسی اختلاف در ارقام اعشار
                if prob_i_str != prob_j_str:
                    diff_count += 1  # شمارش اختلاف در احتمال

            # ذخیره اختلافات در هر دو موقعیت (i, j) و (j, i) برای متقارن بودن
            deltas[i, j] = diff_count
            deltas[j, i] = diff_count

    return deltas

# اجرای تابع با تعیین تعداد ارقام اعشار
diff_score = delta_score_d1(client_models_m, x_sample, decimal_places=1)
print("Distance Matrix for Delta Score:")
print(diff_score)


Distance Matrix for Delta Score:
[[ 0. 19. 19. 18. 18.]
 [19.  0.  4.  4.  4.]
 [19.  4.  0.  4.  5.]
 [18.  4.  4.  0.  3.]
 [18.  4.  5.  3.  0.]]


In [21]:
def delta_score_d2(models, x_test, decimal_places):
    num_models = len(models)
    deltas = np.zeros((num_models, num_models))

    # پیش‌بینی خروجی‌های هر مدل
    predictions = [model.predict(x_test) for model in models]
    
    # محاسبه‌ی argmax برای هر مدل
    argmax_preds = [np.argmax(pred, axis=1) for pred in predictions]

    # مقایسه مدل‌ها دو به دو
    for i in range(num_models):
        for j in range(i + 1, num_models):
            # مقایسه‌ی argmax ها
            argmax_diff = argmax_preds[i] != argmax_preds[j]
            diff_count = np.sum(argmax_diff)  # شمارش اختلاف در argmax
            
            # مقایسه‌ی احتمالات اگر argmax یکسان باشد
            same_argmax_indices = np.where(argmax_preds[i] == argmax_preds[j])[0]
            
            for idx in same_argmax_indices:
                # مقایسه‌ی احتمالات در همان کلاس‌های argmax
                prob_i = predictions[i][idx][argmax_preds[i][idx]]
                prob_j = predictions[j][idx][argmax_preds[j][idx]]
                
                # استخراج رشته‌ی ارقام اعشار تا تعداد مشخص‌شده
                prob_i_str = str(prob_i)[:decimal_places + 2]  # 2 برای "0."
                prob_j_str = str(prob_j)[:decimal_places + 2]

                # بررسی اختلاف در ارقام اعشار
                if prob_i_str != prob_j_str:
                    diff_count += 1  # شمارش اختلاف در احتمال

            # ذخیره اختلافات در هر دو موقعیت (i, j) و (j, i) برای متقارن بودن
            deltas[i, j] = diff_count
            deltas[j, i] = diff_count

    return deltas

# اجرای تابع با تعیین تعداد ارقام اعشار
diff_score = delta_score_d2(client_models_m, x_sample, decimal_places=2)
print("Distance Matrix for Delta Score:")
print(diff_score)


Distance Matrix for Delta Score:
[[ 0. 19. 19. 18. 18.]
 [19.  0.  7.  6.  7.]
 [19.  7.  0.  9.  9.]
 [18.  6.  9.  0.  5.]
 [18.  7.  9.  5.  0.]]


In [12]:
from scipy.stats import ks_2samp

def p_ks(models, x_test):
    n_models = len(models)
    n_samples = x_test.shape[0]
    
    # پیش‌بینی مدل‌ها برای x_test
    model_predictions = [model.predict(x_test) for model in models]
    
    # ساخت ماتریس متقارن برای نتایج KS
    ks_matrix = np.zeros((n_models, n_models))
    
    # بررسی تک تک نمونه‌ها
    for i in range(n_samples):
        # پیدا کردن برچسب اکثریت برای این نمونه
        majority_labels = [np.argmax(pred[i]) for pred in model_predictions]
        majority_label = np.bincount(majority_labels).argmax()

        # گرفتن احتمال هر مدل برای برچسب اکثریت
        majority_probs = [pred[i][majority_label] for pred in model_predictions]

        # محاسبه تست KS بین مدل‌ها
        for m1 in range(n_models):
            for m2 in range(m1 + 1, n_models):
                ks_stat, _ = ks_2samp([majority_probs[m1]], [majority_probs[m2]])
                ks_matrix[m1, m2] += ks_stat
                ks_matrix[m2, m1] = ks_matrix[m1, m2]  # ماتریس متقارن

    # تقسیم بر تعداد نمونه‌ها برای نرمال‌سازی
    # ks_matrix /= n_samples
    return ks_matrix

diff_ks = p_ks(client_models_m, x_sample)
print("Distance Matrix for KS:")
print(diff_ks)

Distance Matrix for KS:
[[ 0. 19. 19. 18. 18.]
 [19.  0. 19. 18. 19.]
 [19. 19.  0. 19. 19.]
 [18. 18. 19.  0. 18.]
 [18. 19. 19. 18.  0.]]


In [26]:
from scipy.stats import chisquare

def p_x2(models, x_test):
    # تعداد مدل‌ها
    num_models = len(models)

    # پیش‌بینی برچسب‌ها توسط مدل‌ها
    predictions = np.array([model.predict(x_test) for model in models])  # shape: (num_models, num_samples, num_classes)

    # اطمینان از اینکه پیش‌بینی‌ها به صورت دسته‌ای هستند
    if len(predictions.shape) != 3:
        raise ValueError("Predictions must be a 3D array with shape (num_models, num_samples, num_classes).")

    # برچسب اکثریت برای هر نمونه
    majority_labels = np.argmax(predictions, axis=2)  # shape: (num_models, num_samples)

    # محاسبه احتمال هر برچسب
    probabilities = np.array([[np.mean(majority_labels[i] == j) for j in range(np.max(majority_labels) + 1)]
                              for i in range(num_models)])  # shape: (num_models, num_classes)

    # اضافه کردن مقدار کوچک برای جلوگیری از تقسیم بر صفر
    probabilities += 1e-10  # جلوگیری از صفر در احتمال‌ها

    # محاسبه ماتریس Chi-square
    x2_distance_matrix = np.zeros((num_models, num_models))

    for i in range(num_models):
        for j in range(i + 1, num_models):
            # استفاده از تست Chi-square برای محاسبه فاصله
            stat, p = chisquare(probabilities[i], f_exp=probabilities[j])
            x2_distance_matrix[i, j] = stat
            x2_distance_matrix[j, i] = stat  # ماتریس متقارن

    return x2_distance_matrix
    
# Example usage
diff_x2 = p_x2(client_models_m, x_sample)
print("Distance Matrix for x2:")
print(diff_x2)


Distance Matrix for x2:
[[0.0e+00 1.0e+08 1.0e+08 1.0e+08 1.0e+08]
 [1.0e+08 0.0e+00 3.5e-02 3.5e-02 3.5e-02]
 [1.0e+08 3.5e-02 0.0e+00 0.0e+00 0.0e+00]
 [1.0e+08 3.5e-02 0.0e+00 0.0e+00 0.0e+00]
 [1.0e+08 3.5e-02 0.0e+00 0.0e+00 0.0e+00]]


### problam client

In [27]:
def meancal(matrix):
    temp = 0
    x = matrix.shape[0]    
    arrmean = []
    
    for i in range(0,x):
        #print(matrix[i].mean())
        temp = ((matrix[i].mean())*10)/9
        arrmean.append(temp)
    return arrmean

def iqrfunc(nparray, multiplier = 1.5):
    data = np.array(nparray)
    q1 = np.percentile(data,25)
    q3 = np.percentile(data,75)
    iqr = q3 -q1
    lower_bound = q1-(multiplier*1.5)
    upper_bound = q3+(multiplier*1.5)
    outliers = np.where((data<lower_bound) | (data>upper_bound))[0]
    return outliers
    
diff_class = delta_class(client_models_m, x_test_m)
diff_score1 = delta_score_d1(client_models_m, x_test_m,1)
diff_score2 = delta_score_d2(client_models_m, x_test_m,2)
diff_score_nd = delta_score_nd(client_models_m, x_test_m)
diff_score_av = delta_score_av(client_models_m, x_test_m)
diff_ks = p_ks(client_models_m, x_test_m)
diff_x2 = p_x2(client_models_m, x_test_m)
# diff_x2 = delta_x2(client_models_m, x_test_m)

temp_c = meancal(diff_class)
temp_s1 = meancal(diff_score1)
temp_s2 = meancal(diff_score2)
temp_snd = meancal(diff_score_nd)
temp_sav = meancal(diff_score_av)
temp_ks = meancal(diff_ks)
temp_x2 = meancal(diff_x2)
# temp_x2 = meancal(diff_score)
print(f"Problemmatic Clients for Delta class {iqrfunc(temp_c)}:")                       
print(f"Problemmatic Clients for Delta Score Decimal1 {iqrfunc(temp_s1)}:")                       
print(f"Problemmatic Clients for Delta Score Decimal2 {iqrfunc(temp_s2)}:")                       
print(f"Problemmatic Clients for Delta Score No Decimal {iqrfunc(temp_snd)}:")                       
print(f"Problemmatic Clients for Delta Score Actual value {iqrfunc(temp_sav)}:")                       
print(f"Problemmatic Clients for KS {iqrfunc(temp_ks)}:")                       
print(f"Problemmatic Clients for X2 {iqrfunc(temp_x2)}:")                       


NameError: name 'delta_score_d2' is not defined

### Ahanin code's

In [15]:
def delta_class(model1, model2, x_test):
    # Make predictions on the test data
    predictions1 = np.argmax(model1.predict(x_test), axis=1)
    predictions2 = np.argmax(model2.predict(x_test), axis=1)
    
    # Count the number of differing predictions
    count = np.sum(predictions1 != predictions2)
    
    return count

# Initialize the distance matrix
num_clients = len(client_models_m)
distance_matrix = np.zeros((num_clients, num_clients))

# Calculate the delta class for each pair of models
for i in range(num_clients):
    for j in range(i + 1, num_clients):
        distance_matrix[i, j] = delta_class(client_models_m[i], client_models_m[j],x_sample)
        distance_matrix[j, i] = distance_matrix[i, j]  # Symmetric matrix

# Print the distance matrix
print("Distance Matrix:")
print(distance_matrix)

Distance Matrix:
[[0. 3. 2. 2. 2.]
 [3. 0. 1. 1. 1.]
 [2. 1. 0. 0. 0.]
 [2. 1. 0. 0. 0.]
 [2. 1. 0. 0. 0.]]


In [14]:
def delta_score(model1, model2, x_test, threshold=0.1):
    # Make predictions on the test data
    predictions1 = model1.predict(x_test)
    predictions2 = model2.predict(x_test)
    
    # Get the predicted labels and their probabilities
    labels1 = np.argmax(predictions1, axis=1)
    labels2 = np.argmax(predictions2, axis=1)
    
    count = 0
    for i in range(len(labels1)):
        # Check if the predicted labels are different
        if labels1[i] != labels2[i]:
            count += 1
        else:
            # If labels are the same, compare the probabilities rounded to 1 decimal place
            prob1 = round(predictions1[i][labels1[i]], 1)
            prob2 = round(predictions2[i][labels2[i]], 1)
            if abs(prob1 - prob2) > threshold:  # Compare probabilities
                count += 1
    
    return count

# Initialize the distance matrix
num_clients = len(client_models_m)
delta_score_matrix = np.zeros((num_clients, num_clients))

# Calculate the delta score for each pair of models
for i in range(num_clients):
    for j in range(i + 1, num_clients):
        delta_score_matrix[i, j] = delta_score(client_models_m[i], client_models_m[j], x_sample)
        delta_score_matrix[j, i] = delta_score_matrix[i, j]  # Symmetric matrix

# Print the delta score matrix
print("Delta Score Matrix:")
print(delta_score_matrix)


Delta Score Matrix:
[[0. 5. 4. 4. 3.]
 [5. 0. 5. 3. 4.]
 [4. 5. 0. 4. 5.]
 [4. 3. 4. 0. 3.]
 [3. 4. 5. 3. 0.]]


In [13]:
from scipy.stats import ks_2samp

def ks_test(model1, model2, x_test):
    # Make predictions on the test data
    predictions1 = model1.predict(x_test)
    predictions2 = model2.predict(x_test)
    
    # Initialize a list to store KS statistics for each class
    ks_stats = []
    
    # Perform KS test for each class and store the KS statistic
    for i in range(predictions1.shape[1]):
        ks_stat, _ = ks_2samp(predictions1[:, i], predictions2[:, i])
        ks_stats.append(ks_stat)
    
    return ks_stats

# Initialize a matrix to store KS test results
num_clients = len(client_models_m)
ks_stat_matrix = np.zeros((num_clients, num_clients))

# Calculate the KS test for each pair of models
for i in range(num_clients):
    for j in range(i + 1, num_clients):
        ks_stats = ks_test(client_models_m[i], client_models_m[j], x_test_m)
        ks_stat_matrix[i, j] = np.mean(ks_stats)
        ks_stat_matrix[j, i] = ks_stat_matrix[i, j]  # Symmetric matrix

# Print the KS test statistic matrix
print("KS Test Statistic Matrix:")
print(ks_stat_matrix)

KS Test Statistic Matrix:
[[0.      0.89507 0.89606 0.89517 0.89534]
 [0.89507 0.      0.0793  0.07589 0.08482]
 [0.89606 0.0793  0.      0.09489 0.10277]
 [0.89517 0.07589 0.09489 0.      0.06699]
 [0.89534 0.08482 0.10277 0.06699 0.     ]]


In [23]:
from scipy.stats import chi2_contingency

def chi_square_test(model1, model2, x_test):
    # Make predictions on the test data
    predictions1 = np.argmax(model1.predict(x_test), axis=1)
    predictions2 = np.argmax(model2.predict(x_test), axis=1)
    
    # Create a contingency table
    contingency_table = np.zeros((10, 10))
    for i in range(len(predictions1)):
        contingency_table[predictions1[i], predictions2[i]] += 1
    
    # Perform Chi-Square test
    chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)
    
    return chi2_stat

# Initialize a matrix to store Chi-Square test results
num_clients = len(client_models_m)
chi2_stat_matrix = np.zeros((num_clients, num_clients))

# Calculate the Chi-Square test for each pair of models
for i in range(num_clients):
    for j in range(i + 1, num_clients):
        chi2_stat = chi_square_test(client_models_m[i], client_models_m[j], x_test_m)
        chi2_stat_matrix[i, j] = chi2_stat
        chi2_stat_matrix[j, i] = chi2_stat  # Symmetric matrix

# Print the Chi-Square test statistic matrix
print("Chi-Square Test Statistic Matrix:")
print(chi2_stat_matrix)

Chi-Square Test Statistic Matrix:
[[    0.         65999.36986572 66069.9587529  65416.70235105
  66681.73215678]
 [65999.36986572     0.         81534.16052464 81571.11264982
  82840.61392205]
 [66069.9587529  81534.16052464     0.         81773.43585289
  82352.77255414]
 [65416.70235105 81571.11264982 81773.43585289     0.
  82065.7629604 ]
 [66681.73215678 82840.61392205 82352.77255414 82065.7629604
      0.        ]]
